In [1]:
from optimization_func import simulated_annealing
from helper import mse, mae, r2, lotka_volterra, int_cost_lotka_volterra, get_predator_prey_data
t, P = get_predator_prey_data()
import shelve
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy import stats


In [2]:
mpl.rcParams["figure.dpi"] = 180
mpl.rcParams["legend.fontsize"] = 11
plt.rc("text", usetex=True)
plt.rc("text.latex", preamble=r"""
\usepackage{palatino}
\usepackage{newpxmath}""")
plt.rc("font", family="serif")

In [3]:
def sp_confidence_interval(samples, p=0.95):
    """ Calculates the confidence interval (with p=95% unless noted otherwise) 
    Returns:
         Confidence radius (int) based on samples.     
    """
    a = stats.t.interval(p, df=len(samples)-1, loc=np.mean(samples), scale=stats.sem(samples))
    return a[1] - a[0]

In [8]:
def Convergence_data(rvs, cost, n_sim=100, gamma = ?):
    Dict = {}
    for rv in rvs:
        sim_data = []
        r2s = []
        for n in range(n_sim):
            state, _, _, costs = simulated_annealing(rv, t, P, cost=cost, cooling_schedule="quadratic", T_start=200, T_steps=2000, write_costs = True, gamma = gamma)
            sim_data.append(costs)
            r2_value = int_cost_lotka_volterra(state, P, t, cost=r2)
            r2s.append(r2_value)
        Dict[str(rv)] = (sim_data, r2s)
    
    return Dict           
    
    

In [9]:
xs = [[0.66593614, 2.94213991, 8.129154  , 4.88824876],
       [3.13320535, 0.61170786, 1.20556467, 8.54715741],
       [1.59035578, 3.964897  , 8.93986609, 2.72514184],
       [4.73006778, 6.44811731, 5.9776969 , 7.29995916],
       [1.72004538, 8.47855451, 1.16241656, 0.58402861],
       [5.14251217, 2.59650137, 3.03574317, 7.63356504],
       [6.07600054, 4.30131415, 4.60656356, 3.67890247],
       [3.87088546, 7.40314946, 9.84267204, 5.05488566],
       [5.49825296, 0.65119381, 7.9734132 , 9.63343394],
       [7.26991767, 9.8089839 , 8.61680355, 3.66975829]]

xs2 = [[1000, 50, 30, 1]]



In [10]:
%%time
np.random.seed(567567)
if False: 
    shelf = shelve.open("Arrays/MSE_vs_MAE")
    Test_mse = Convergence_data(rvs=xs[0:2], cost=mse, n_sim=3)
    shelf["Test_mse"] = Test_mse
    Test_mae = Convergence_data(rvs=xs[0:2], cost=mae, n_sim=3)
    shelf["Test_mae"] = Test_mae
shelf.close() 

C:\Users\malou\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\Users\malou\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\Users\malou\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\Users\malou\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\U

Wall time: 2min 12s


In [11]:
def Convergence_plotter(rvs, Test_mse, Test_mae, T_steps=2000):
    iterations = range(1,T_steps + 1)
    for data in  [Test_mse, Test_mae]:
        for rv in rvs:
            data[str(rv)] = results
            np.mean(results, axis = 1)
            a = [sp_confidence_interval(samples) for samples in np.array(results)]
            plt.plot(iterations, mean, 'o', markersize = 2, linewidth=1, color = colors[i])
            plt.errorbar(iterations, mean, yerr=a, color = colors[i], capsize = 1, elinewidth=0.5, markeredgewidth=1)
            
    plt.grid(alpha=0.3)
    plt.ylabel("MSE")
    plt.xlabel("Point in Markov Chain ($k_i$)")
    #plt.legend(["A", ])
    plt.show()         
    
        

run  1
0.11705196219897132
0.15721778296361313
5.895369845548445
7.994762717411582
5.899227223785397
0.10694290387391384
0.0740823219706136
0.24005691068457988
0.039073895252362136
0.06686137801580821
run  2
0.12274024343353618
0.03840839686968239
8.24485339725429
0.1758817321741763
5.898994599415021
0.08951896184823209
8.279241836657445
0.07203979040103733
8.250162444455809
0.47010737390418933


8.244732441935538
